<a href="https://colab.research.google.com/github/khushbooekhande/WildfireDetectionUsingMachineLearning/blob/master/Final_Identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import math
import cv2
from sklearn.externals import joblib 

fireclassifier = load_model('fireclassifier.h5')
smokeclassifier = load_model('smokeclassifier.h5')
randomForest = joblib.load('randomForest.pkl') 
decisionTree = joblib.load('decisionTree.pkl')


Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


OSError: ignored

In [0]:
def smoke_predict(imgage_path):
    test_image = image.load_img(img_path, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = smokeclassifier.predict(test_image)
    if result[0][0] == 1:
        prediction = 'notsmoke'
        return False
    else:
        prediction = 'smoke'
        return True

def smoke_predict_part(inp_arr_image):
    test_image = cv2.resize(inp_arr_image, (64,64))
    test_image = np.expand_dims(test_image, axis = 0)
    result = smokeclassifier.predict(test_image)
    if result[0][0] == 1:
        prediction = 'notsmoke'
        return False
    else:
        prediction = 'smoke'
        return True
    
def fire_predict(imgage_path):
    test_image = image.load_img(img_path, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = fireclassifier.predict(test_image)
    if result[0][0] == 1:
        prediction = 'notfire'
        return False
    else:
        prediction = 'fire'
        return True

def fire_predict_part(inp_arr_image):
    test_image = cv2.resize(inp_arr_image, (64,64))
    test_image = np.expand_dims(test_image, axis = 0)
    result = fireclassifier.predict(test_image)
    if result[0][0] == 1:
        prediction = 'notfire'
        return False
    else:
        prediction = 'fire'
        return True

def plot_image(img_array):
    img = np.array(img_array, np.int32)
    plt.imshow(img)
    

def get_cells_img(np_arr_img, n=64): # considers all n X n grids
    sub_imgs = []
    for row in range((np_arr_img.shape[0]//n)):
        for col in range((np_arr_img.shape[1]//n)):
            c_0 = col*n
            c_1 = min((c_0+n), np_arr_img.shape[1])
            r_0 = row*n
            r_1 = min((r_0+n), np_arr_img.shape[0])
    #         print(c_0, c_1, " | ", r_0, r_1)
            sub_imgs.append(np_arr_img[r_0:r_1, c_0:c_1,: ])
    return sub_imgs

def get_img_array(path):
    img = image.load_img(path)
    return image.img_to_array(img)

def plot_images(img, labels, nrows, ncols):
    fig, axes = plt.subplots(nrows, ncols)
    print(len(axes.flat))
    for i, ax in enumerate(axes.flat): 
        ax.imshow(img[i])
        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title(labels[i])

def count_subimgs(img_arr_pred):
    cnt = 0
    for s in img_arr_pred:
            if s:
                cnt += 1

    return cnt


In [0]:
def classify_fire(test_img):
    test_img_arr = get_img_array(test_img)
    test_img_arr = image.img_to_array(test_img_arr)
    test_img_arr = cv2.resize(test_img_arr, (640, 640) )
    test_img_arr_cells = get_cells_img(test_img_arr, n=128)
    test_img_arr_pred = [fire_predict_part(img) for img in test_img_arr_cells]

    count_yes = count_subimgs(test_img_arr_pred)
    ys_prob = (count_yes/len(test_img_arr_pred))*100
    if (ys_prob > 20):
        return True
    else:
        return False
    
def classify_smoke(test_img):
    test_img_arr = get_img_array(test_img)
    test_img_arr = image.img_to_array(test_img_arr)
    test_img_arr = cv2.resize(test_img_arr, (640, 640) )
    test_img_arr_cells = get_cells_img(test_img_arr, n=128)
    test_img_arr_pred = [smoke_predict_part(img) for img in test_img_arr_cells]

    count_yes = count_subimgs(test_img_arr_pred)
    ys_prob = (count_yes/len(test_img_arr_pred))*100
    if (ys_prob > 20):
        return True
    else:
        return False
    
def detect(img):
    fire = classify_fire(img)
    smoke = classify_smoke(img)
    
    if(fire):
        if(smoke):
#             print("Fire With Smoke")
            return [1, 1]
        else:
#             print("Fire With NO Smoke")
            return [1, 0]
    else:
#         print("No Fire Detected")
        return [0, 0]
    


def final_output(img, data_arr):
    detection = detect(img)
    fire = detection[0]
    smoke = detection[1]
    intensity = predicted_intensity = randomForest.predict(np.array(data_arr).reshape(1,-1))
    intensity = math.ceil(intensity[0])
    if(fire):
        if(smoke):
            print("Fire with Smoke having intensity level " + str(intensity))
        else:
            print("Fire with NO Smoke having intensity level " + str(intensity))
    else:
        print("No Fire ")


In [0]:
final_output('test_img.jpg',[435600.0, 90, 9.0])


Fire with Smoke having intensity level 5
